## Dataset
#### https://www.kaggle.com/datasets/yasserh/walmart-dataset

1. Understand the Dataset & cleanup (if required).
2. Build Regression models to predict the sales w.r.t single & multiple features.
3. Also evaluate the models & compare their respective scores like R2, RMSE, etc.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import plotly.express as py
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()

In [3]:
df = pd.read_csv('Walmart.csv')
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106


In [4]:
df.shape

(6435, 8)

In [5]:
df['Date'] = pd.to_datetime(df['Date'], format="%d-%m-%Y", errors='coerce')
df.head(1)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,2010-02-05,1643690.9,0,42.31,2.572,211.096358,8.106


### Which store has maximum sales

In [6]:
max_value = df.groupby('Store')['Weekly_Sales'].sum()
print("Store {} has maximum sales. Maximum sales is {}".format(max_value.idxmax(),max_value.max()))

Store 20 has maximum sales. Maximum sales is 301397792.46


### Which store/s has good quarterly growth rate in Q3’2012

In [7]:
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

In [8]:
def quarterData(month):
    if month <= 3:
        return 'Q1'
    elif month > 3 and month <= 6:
        return 'Q2'
    elif month > 6 and month <= 9:
        return 'Q3'
    else:
        return 'Q4'
    
df['Quarter'] = df['Date'].dt.month.apply(quarterData)
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Quarter
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106,2010,2,Q1
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106,2010,2,Q1
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,2010,2,Q1
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,2010,2,Q1
4,1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106,2010,3,Q1


### Some holidays have a negative impact on sales. Find out holidays which have higher sales than the mean sales in non-holiday season for all stores together

In [9]:
mean_non_holiday = df[df['Holiday_Flag'] == 0]['Weekly_Sales'].mean()
print('Mean Sales for non holiday ' + str(mean_non_holiday))

Mean Sales for non holiday 1041256.3802088555


In [10]:
neg_holiday_data = df[(df['Holiday_Flag'] == 1) & (df['Weekly_Sales'] < mean_non_holiday)]
neg_holiday_data[neg_holiday_data['Store'] == 3] #.groupby('Store')['Weekly_Sales'].count()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Quarter
287,3,2010-02-12,420728.96,1,47.93,2.548,214.574792,7.368,2010,2,Q1
317,3,2010-09-10,352260.97,1,80.84,2.565,214.806543,7.346,2010,9,Q3
328,3,2010-11-26,565567.84,1,68.71,2.735,215.061403,7.564,2010,11,Q4
333,3,2010-12-31,382677.76,1,53.20,2.943,214.698647,7.564,2010,12,Q4
339,3,2011-02-11,430526.21,1,43.57,3.022,216.266091,7.551,2011,2,Q1
369,3,2011-09-09,377347.49,1,81.72,3.546,219.213530,7.567,2011,9,Q3
380,3,2011-11-25,556925.19,1,68.00,3.236,221.901118,7.197,2011,11,Q4
385,3,2011-12-30,410553.88,1,48.29,3.129,223.009084,7.197,2011,12,Q4
391,3,2012-02-10,473292.47,1,51.65,3.409,223.753643,6.833,2012,2,Q1
421,3,2012-09-07,408229.73,1,84.99,3.730,225.966026,6.334,2012,9,Q3


In [11]:
neg_data = neg_holiday_data.groupby(['Store'])['Weekly_Sales'].aggregate(['min', 'max', 'sum', 'count']).sort_values(by=['count', 'sum'], ascending=False).reset_index()
neg_data.head(1)

,Store,min,max,sum,count
0,7,524104.92,949075.87,6724002.65,10


In [12]:
neg_data.iplot(kind='bar', x='Store')

In [13]:
# neg_holiday_data['Quarter'].iplot("bar")
neg_holiday_data.iplot(kind='bar', x=['Quarter', 'Year'], y='Weekly_Sales')

In [14]:
neg_holiday_data.head(1)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Quarter
287,3,2010-02-12,420728.96,1,47.93,2.548,214.574792,7.368,2010,2,Q1


In [15]:
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Quarter
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106,2010,2,Q1
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106,2010,2,Q1
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,2010,2,Q1
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,2010,2,Q1
4,1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106,2010,3,Q1


In [16]:
# store = df[df['Store', 'Weekly_Sales', 'Holiday_Flag', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Year', 'Month']]
store = df.drop(['Date', 'Month'], axis=1).groupby(['Store'])['Weekly_Sales', 'Fuel_Price', 'CPI'].sum()
store.head(1)

/var/folders/q4/m_b1xrxn53j9tp7kn4drjl080000gn/T/ipykernel_52280/4014760761.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,Weekly_Sales,Fuel_Price,CPI
Store,,,
1,2.224028e+08,460.417,30887.555523
